In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import copy
#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns

init_notebook_mode(connected=True)
cf.go_offline()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator
pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

In [2]:
from sklearn.feature_selection import SelectKBest, VarianceThreshold

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from joblib import dump, load
from Logger import RegressionLogger

In [3]:
log_transformed = 0
class FuncTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self):
        return self
    
    def transform(self, X, y=None):
        return X
        
    def log_transform(self, X, column):
        col = X[:, column]
        print(col.dtype)
        return np.log(col)
        
    
    def exp_transform(self, X, column):
        col = X[:, log_transformed]
        return np.exp(col)
        
    
    def poly_transform(self, X, y, column, degree):
        col = X[:, column]
        p1 = np.polyfit(col, y, degree)
        return np.polyval(p1, col)
        
    
    def recip_transform(self, X, column):
        col = X[:, column]
        return 1 / col
        
        
    
        

In [4]:
model_log = load("data/model_logging.joblib")

In [5]:
current_model = copy.deepcopy(model_log[0]["model"])

In [6]:
params = {
    "Imputer": "Simple Imputer",
    "Scaler": "Standard Scaler",
    "model_name": "Elastic Net",
    "model": current_model,
    "alpha": 0.1,
    "l1_ratio": 0.5,
}

In [7]:
logger = RegressionLogger(params)

# Now Lets Load our original Data

In [8]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")

In [9]:
df.drop("Unnamed: 0",axis=1, inplace=True)

In [10]:
df.drop(["City location of institution (HD2019)", "Institution Name"], axis=1, inplace=True)

In [11]:
X = df.iloc[:, :-14]
y = df.iloc[:,-14]

In [12]:
df_black = df.dropna(subset=[df.columns[-7]])

In [13]:
X_black = df_black.iloc[:,:-14]
y_black = df_black.iloc[:, -7]

In [14]:
X_black = pd.get_dummies(X_black, drop_first=True)

In [15]:
X = pd.get_dummies(X, drop_first=True)

In [16]:
X.shape

(2315, 260)

In [17]:
X_black.shape

(2027, 255)

# Lets test against different imputation strategies: 
<ol>
    <li>SimpleImputer</li>
    <li>KNNImputer()</li>
    <li>IterativeImputer()</li>
    <li>Let XG Boost handle imputation</li>
<ol>

### KNN imputation shows improvement so lets try iterative

In [18]:
X.shape

(2315, 260)

In [19]:
# Reset Current Model
current_model = copy.deepcopy(model_log[0]["model"])

In [20]:
num_cols = list(range(len(X.select_dtypes(include=["float"]).columns)))

In [21]:
num_cols

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121]

In [22]:
#update model pipeline 
current_model.steps[0] = ('Imputer', ColumnTransformer(
    [
        ('Impute', KNNImputer(), num_cols)
    ], remainder="passthrough"
))

In [23]:
#define new parameters
params = {
    "Imputer": "KNNImputer",
    "Scaler": "StandardScaler",
    "model": current_model,
    "model_name": "Elastic_Net",
    "alpha": 0.1,
    "l1_ratio": 0.5,   
}

In [24]:
#reset logger paramters
logger.__init__(params)

In [25]:
#train and record results
logger.train_update(current_model, X, y)
logger.train_update(current_model, X_black, y_black, suffix="_black")
logger.save_log("KNN imputation with Elastic Net")
model_log.append(logger.record())

### KNN shows slightly better results now lets try RandomForestRegressor

In [26]:
# reset the current model
rf = copy.deepcopy(model_log[0]["model"])

In [27]:
# update Pipeline
rf_hyperparams = {
    "n_estimators": 100,
    "max_depth": 15,
    
    "min_samples_split": 10,
    "min_samples_leaf": 5
} # hyperparameters for XGBoost


In [28]:
rf_params = {
    "model": rf,
    "model_name": "Random Forest",
    
    "n_estimators": 100,
    "max_depth": 15,
    "max_features": 'auto',
    "min_samples_split": 10,
    "min_samples_leaf": 5
}

In [29]:
rf.steps[2] = ('classifier', RandomForestRegressor(**rf_hyperparams))

In [30]:
#reset logger paramters
logger.__init__(rf_params)

In [31]:
#train and record
logger.train_update(rf, X, y)
logger.train_update(rf, X_black, y_black, "_black")
logger.save_log("Random Forest Tree")
model_log.append(logger.record())

# Tree Based Model Performed Much Better; Lets Try XGB tree

In [32]:
# Reset Current Model
xgb_model = copy.deepcopy(model_log[0]["model"])

In [33]:
# update Pipeline
params = {
    "objective": "reg:squarederror",
    "booster": "gbtree",
    "learning_rate": 0.1,
    "subsample": 1.0,
    "min_child_weight": 1
} # hyperparameters for XGBoost

In [34]:
# update pipeline
xgb_model.steps.pop(0)

xgb_model.steps[1] = ('classifier', xgb.XGBRegressor(**params))

In [35]:
# all hyperparameters for logger
xgb_hyperparams = {
    "model": xgb_model,
    "model_name": "XGBoost Tree",
    
    "objective": "reg:squarederror",
    "booster": "gbtree",
    "learning_rate": 0.1,
    "subsample": 1.0,
    "min_child_weight": 1
}

In [36]:
#reset logger paramters
logger.__init__(xgb_hyperparams)

In [37]:
X_black.shape

(2027, 255)

In [38]:
#train and record
logger.train_update(xgb_model, X_black, y_black, "_black")
logger.train_update(xgb_model, X, y)
logger.save_log("XGBoost with no imputation")
model_log.append(logger.record())

In [39]:
pd.DataFrame(model_log)[["model_name", "mae"]]

,model_name,mae
0,Elastic Net,9.888576
1,Elastic_Net,9.670773
2,Random Forest,9.042008
3,XGBoost Tree,8.758335


# Lets Do Feature Engineering and try to get the most accurate model we can 

In [40]:
# create column y_actual and residuals as a df

y_hat = logger.model.predict(X)
resid = y - y_hat
target_df = pd.concat([y, resid], axis=1)
target_df.columns = ["Graduation_Rate", "Residuals"]

# Financial Features

In [41]:

picker1 = wg.Dropdown(
    options=X.columns[0:25],
    value=X.columns[0],
    description='Feature:',
    disabled=False,
)

picker2 = wg.Dropdown(
    options=target_df.columns,
    description='Target:',
    value=target_df.columns[-1],
    disabled=False,
)

ui = wg.HBox([picker1, picker2])

def scatter_residuals(feature, target):
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=([feature, "Residuals"]),
        column_widths=[0.6, 0.4]
    
    )
    
    _ = fig.add_trace(go.Scatter(
        x = X[feature], y=target_df[target], mode='markers'
    ), 1, 1)

    _ = fig.add_trace(go.Scatter(
        x = X[feature], y=target_df["Residuals"], mode='markers'
    ), 1, 2)
    
    _ = display(fig.show())
    
out = wg.interactive_output(scatter_residuals, {'feature': picker1, 'target':picker2})
out.layout.height = '700px'
_ = display(ui, out)

Output(layout=Layout(height='700px'))

## Feature Engineering Ideas

<ol>
    <li>Log all dollar amounts</li>
    <li>PCA on distribution of revenues and distribution of expenses</li>
<ol>

In [42]:
# log of core revenues
transformer = FuncTransformer()

In [43]:
fe_x = transformer.log_transform(X.to_numpy(), 0)

float64


In [44]:
X["log_core_revenues"] = fe_x

In [48]:
X

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und


# Staff Features

In [46]:


picker1 = wg.Dropdown(
    options=X.filter(regex="staff").columns,
    value=X.filter(regex="staff").columns[0],
    description='Feature:',
    disabled=False,
)

picker2 = wg.Dropdown(
    options=target_df.columns,
    description='Target:',
    value=target_df.columns[-1],
    disabled=False,
)

ui = wg.HBox([picker1, picker2])

def scatter_residuals(feature, target):
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=([feature, "Residuals"]),
        column_widths=[0.6, 0.4]
    
    )
    
    _ = fig.add_trace(go.Scatter(
        x = X[feature], y=target_df[target], mode='markers'
    ), 1, 1)

    _ = fig.add_trace(go.Scatter(
        x = X[feature], y=target_df["Residuals"], mode='markers'
    ), 1, 2)
    
    _ = display(fig.show())
    
out = wg.interactive_output(scatter_residuals, {'feature': picker1, 'target':picker2})
out.layout.height = '700px'
_ = display(ui, out)

Output(layout=Layout(height='700px'))

<ol>
    <li>Log all dollar amounts</li>
    <li>PCA on distribution of revenues and distribution of expenses</li>
<ol>

### Log transformation for core revenues and expenses should help our model. Dollar amounts show the same trend

In [47]:
fig = make_subplots( 
    rows=2, cols=2,
    subplot_titles = ["Tuition and Fees", "Residuals", "Government_Grants", "Residuals"]
)

fig.add_trace(
    go.Scatter(
        x=X.Tuition_And_Fees, y=y, mode='markers',
        range_x=[0,100]
    )
)


ValueError: Invalid property specified for object of type plotly.graph_objs.Scatter: 'range'

Did you mean "line"?

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            customdata .
        dx
            Sets the x coordinate step